In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

____
## Tags: #Tables

# Links:

[[t_full_citizen_id_in_flat_with_subscriptions]]

In [5]:
query_text = """--sql
    CREATE TABLE db1.t_citizen_id_in_flat_with_subscriptions
    (
        `report_date` Date,
        `address_uuid` String,
        `citizen_id_in_flat_with_subscriptions` UInt64
    )
    ENGINE = MergeTree()
    ORDER BY address_uuid
    """
ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

In [ ]:
query_text = """--sql
CREATE MATERIALIZED VIEW db1.t_citizen_id_in_flat_with_subscriptions_mv
REFRESH EVERY 1 DAY OFFSET 5 HOUR 27 MINUTE TO db1.t_citizen_id_in_flat_with_subscriptions AS 
SELECT
    report_date,
    address_uuid,
    COUNT(DISTINCT citizen_id) as citizen_id_in_flat_with_subscriptions
FROM db1.t_full_citizen_id_in_flat_with_subscriptions
WHERE address_uuid !=''
GROUP BY report_date,
        address_uuid
    """
ch.query_run(query_text)

In [ ]:
start_date = datetime.datetime.strptime('2023-09-01','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2025-09-05','%Y-%m-%d').date()

dates_pd = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d'),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('%/year=%-Y/month=%m/%d%')
        })
dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
for day_index in dates_pd.index:
    date = dates_pd.loc[day_index,['date']].values[0]
    query_text = f"""
        INSERT INTO db1.t_citizen_id_in_flat_with_subscriptions
        SELECT
            report_date,
            address_uuid,
            COUNT(DISTINCT citizen_id) as citizen_id_in_flat_with_subscriptions
        FROM db1.t_full_citizen_id_in_flat_with_subscriptions
        WHERE address_uuid !=''
            AND report_date = '{date}'
        GROUP BY report_date,
                address_uuid
    """
    ch.query_run(query_text)
    print(date)

2025-09-05
2025-09-04
2025-09-03
2025-09-02
2025-09-01
2025-08-31
2025-08-30
2025-08-29
2025-08-28
2025-08-27
2025-08-26
2025-08-25
2025-08-24
2025-08-23
2025-08-21
2025-08-20
2025-08-19
2025-08-18
2025-08-17
2025-08-16
2025-08-15
2025-08-14
2025-08-13
2025-08-12
2025-08-11
2025-08-10
2025-08-09
2025-08-08
2025-08-07
2025-08-06
2025-08-05
2025-08-04
2025-08-03
2025-08-02
2025-08-01
2025-07-31
2025-07-30
2025-07-29
2025-07-28
2025-07-27
2025-07-26
2025-07-25
2025-07-24
2025-07-23
2025-07-22
2025-07-21
2025-07-20
2025-07-19
2025-07-18
2025-07-17
2025-07-16
2025-07-15
2025-07-14
2025-07-13
2025-07-12
2025-07-11
2025-07-10
2025-07-09
2025-07-08
2025-07-07
2025-07-06
2025-07-05
2025-07-04
2025-07-03
2025-07-02
2025-07-01
2025-06-30
2025-06-29
2025-06-28
2025-06-27
2025-06-26
2025-06-25
2025-06-24
2025-06-23
2025-06-22
2025-06-21
2025-06-20
2025-06-19
2025-06-18
2025-06-17
2025-06-16
2025-06-15
2025-06-14
2025-06-13
2025-06-12
2025-06-11
2025-06-10
2025-06-09
2025-06-08
2025-06-07
2025-06-06

In [6]:
query_text = """--sql
SELECT
    *
FROM db1.t_citizen_id_in_flat_with_subscriptions
ORDER BY report_date desc
LIMIT 10
    """
ch.query_run(query_text)

report_date,address_uuid,citizen_id_in_flat_with_subscriptions
date,str,i64
2025-09-06,"""abb20310-b6a2-4c84-850e-473a13…",1
2025-09-06,"""ab9f1003-fc57-46aa-81e0-f2bddf…",11
2025-09-06,"""ab9f82f8-b3ef-44e3-867c-f62780…",4
2025-09-06,"""aba15a32-f93e-479e-8ac2-f474bb…",4
2025-09-06,"""aba5a955-2ca0-4bc6-8c89-e5797b…",3
2025-09-06,"""ab9cff14-6267-4ea2-95d9-987fe7…",47
2025-09-06,"""abaa09aa-d313-4ba7-b9f8-ccac06…",4
2025-09-06,"""abac2898-7e6c-401c-908f-a45462…",6
2025-09-06,"""aba73140-6c29-4449-861a-d5864d…",1


In [4]:
query_text = """
    DROP TABLE db1.t_citizen_id_in_flat_with_subscriptions_mv
    """
ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 60
 Code: 60. DB::Exception: Table db1.t_citizen_id_in_flat_with_subscriptions_mv does not exist. Maybe you meant db1.t_full_citizen_id_in_flat_with_subscriptions_mv?. (UNKNOWN_TABLE) (version 25.3.6.56 (official build))


In [3]:
query_text = """
    DROP TABLE db1.t_citizen_id_in_flat_with_subscriptions
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,1890787,"""f55c4f2c-d191-4c13-bdbc-332af2…"


In [ ]:
query_text = """
    REFRASHE TABLE db1.t_citizen_id_in_flat_with_subscriptions_mv
    """
ch.query_run(query_text)